In [1]:

# Importing the libraries
import configparser
import tweepy
import pandas as pd

import uuid
from datetime import datetime
data_ref=datetime.today().strftime("%Y-%m-%d")

In [2]:
# Read the config file
config = configparser.ConfigParser()
config.read('../config.ini_local')

# Read the values
api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']
access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

In [3]:
# Authenticate
auth = tweepy.OAuth1UserHandler(
   api_key, api_key_secret, access_token, access_token_secret
)

api = tweepy.API(auth)

- Rafael Lange - Cellbit
- Benvenuti - BRKsEDU
- Malena Nunes - Loiranoob

In [4]:
def save_tweets(lista_df, userID):
    df = pd.DataFrame(lista_df)
    df.to_csv(f'../data/influencer_tweets/{userID}_{data_ref}_{str(uuid.uuid4())}.csv', index=False) 

In [5]:
def get_tweets_user(api, userID):
    tweets = api.user_timeline(screen_name=userID, 
                            # 200 is the maximum allowed count
                            count=200,
                            include_rts = False,
                            # Necessary to keep full_text 
                            # otherwise only the first 140 words are extracted
                            tweet_mode = 'extended'
                            )

    lista_df = []

    for info in tweets:
        lista_df.append(
            {
            "id":info.id,
            'userID': userID,
            "created_at":info.created_at,
            "full_text":info.full_text,
            "favorite_count":info.favorite_count,
            "retweet_count":info.retweet_count,
            }
    )
    
    save_tweets(lista_df=lista_df, userID=userID)

In [6]:
def get_user_data(api, userID):
    # Obter informações do usuário
    user = api.get_user(screen_name=userID)

    # Obter a quantidade de seguidores do usuário
    followers_count = user.followers_count

    tweets_count = user.statuses_count

    uui_id = str(uuid.uuid4())

    data_influencer = {
        'uuid': uui_id,
        'data_ref' : data_ref,
        'userID': userID,
        'seguidores' : followers_count,
        'twwets': tweets_count
    }

    pd.DataFrame([data_influencer]).to_csv(f'../data/influencer_data/{userID}_{data_ref}_{uui_id}.csv', index=False)

## Coletando dados dos 3 usuarios

In [7]:
from tqdm import tqdm

In [8]:
userIDList = [
    "Cellbit", "Loiranoob", "BRKsEDU"
]

for i in tqdm(range(200)):
    for user_id in userIDList:
        get_tweets_user(api=api, userID=user_id)

for user_id in userIDList:
    get_user_data(api, user_id)

100%|██████████| 200/200 [07:19<00:00,  2.20s/it]
